# `Clear GPU`
----

In [2]:
import torch

# Clear GPU memory
torch.cuda.empty_cache()

# `Inject .js`
----

In [ ]:
from IPython.display import Javascript, display

def inject_js():
    js_code = """
    function copyToClipboard(button) {
        let cell = button.closest(".cell");  // Find the Jupyter cell container
        let codeBlock = cell.querySelector("pre");  // Get the first <pre> inside this cell

        if (codeBlock) {
            let text = codeBlock.innerText.trim();
            navigator.clipboard.writeText(text).then(() => {
                button.innerText = "✅ Copied!";
                setTimeout(() => button.innerText = "📋 Copy", 2000);
            }).catch(err => console.error("Copy failed:", err));
        } else {
            console.error("No code block found in this cell.");
        }
    }
    """
    display(Javascript(js_code))
inject_js()  # Injects JavaScript into Jupyter

# `Abs Path`
----

In [6]:
from pathlib import Path
import inspect

def _path(f: str | Path, dir: str | Path = Path.cwd()) -> Path:
    """
    Searches for a file by name recursively from a given directory.

    Parameters:
    -----------
    f : str or Path
        The name of the file to search for.
    dir : str or Path, optional
        The directory to start searching from (default: current working directory).

    Returns:
    --------
    Path
        The absolute path of the found file.
    """

    assert dir is Path or dir is str, f'\n\n[INVALID ARGS]: \n\tFunction only accepts type str or Path for <dir> arg.\n\n[INSPECT]:\n\t{_path.__name__}{inspect.signature(_path)}\n'
    
    if isinstance(dir, str):
        dir = Path(dir)
        
    for path in dir.rglob(f): # recursively searches for the file
        return path.resolve() # kicks back first match
    
    raise FileNotFoundError(f"\"I'm sorry Dave. I'm afraid I can't find '{f}' in {dir}\"\n - HAL 9000")

# `Debug`
----

In [2]:
from typing import Callable, TypeVar, Any
import threading

''' CUSTOM DEBUG TOOL FROM MY OWN LIL' LIBRARY OF HELPERS '''

__COLORS = {
        (_DEBUG := "red"): "\033[91m",
        (_HEADING := "green"): "\033[92m",
        (_UPDATE :="blue"): "\033[94m",
        (_RESET := "reset"): "\033[0m"
    }

# thread-local storage to track nested debug states
bugs = _debug_stack = threading.local()
F = TypeVar('F', bound = Callable) # generic function type

def debug(*, enabled: bool = False) -> Callable[[F], F]:
    import functools
    ''' Decorator that enables or disables debug prints inside the wrapped function. '''

    def decorator(func: F) -> F:
        @functools.wraps(func)
        def wrapper(*args: Any, **kwargs: Any) -> Any:

            if not hasattr(_debug_stack, 'state'):
                _debug_stack.nest = [] # init debug state if missing

            _debug_stack.nest.append(enabled) # push current function's debug flag

            if enabled:
                print(f'\n{__COLORS.get(_HEADING)}[START] {__COLORS[_RESET]}')
                print(f'{__COLORS.get(_DEBUG)}[DEBUG] {__COLORS["reset"]}' + 
                      f'{__COLORS.get(_UPDATE)}Calling "{func.__name__}"...{__COLORS.get(_RESET)}')
            
            try:
                result = func(*args, **kwargs)  # call actual function so behavior is as intended
            finally:
                _debug_stack.nest.pop()
            
            if enabled:
                print(f'{__COLORS.get(_DEBUG)}[DEBUG] {__COLORS[_RESET]}' + 
                      f'{__COLORS.get(_UPDATE)}Function "{func.__name__}" exited...{__COLORS.get(_RESET)}')
                print(f'{__COLORS.get(_HEADING)}[END] {__COLORS[_RESET]}\n')         
            
            return result  # make sure the function still behaves as intended
        return wrapper # return wrapped function
    return decorator  # returns the decorator

# `Unzip`
----

In [ ]:
import zipfile
from pathlib import Path
from typing import TypeVar

T = TypeVar("T", str, Path)

def _unzip(zip_path: T, extract_to: T = None) -> None:
    """
    Extracts the contents of a ZIP file to a specified directory.

    If no extraction directory is provided, it defaults to extracting in the 
    same directory as the ZIP file.

    Parameters:
    -----------
    zip_path : str
        The path to the ZIP file to be extracted.
    extract_to : str, optional
        The directory where the files should be extracted. If None, extracts 
        to the same directory as the ZIP file.

    Raises:
    -------
    FileNotFoundError
        If the ZIP file does not exist.
    zipfile.BadZipFile
        If the file is not a valid ZIP archive.
    """
    zip_path = Path(zip_path)

    if not zip_path.exists():
        raise FileNotFoundError(f"ZIP file not found: {zip_path}")

    extract_to = Path(extract_to) if extract_to else zip_path.parent

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    print(f"Extracted '{zip_path.name}' to '{extract_to}'")


if __name__ == "__main__":
    # Example test case for running the script directly
    test_zip = "test.zip"
    test_output = "output_folder"

    try:
        _unzip(test_zip, test_output)
    except FileNotFoundError:
        print(f"Test failed: '{test_zip}' not found. Please create a test ZIP file.")
    except zipfile.BadZipFile:
        print(f"Test failed: '{test_zip}' is not a valid ZIP file.")

# `Find CWD`
----

In [ ]:
from pathlib import Path

def _dir() -> Path:
    '''
    Returns the absolute path of the current script's directory.

    If the function is executed within a standalone Python script, it resolves 
    the directory using `__file__`. If `__file__` is not available (such as in 
    Jupyter Notebook or interactive Python environments), it falls back to the 
    current working directory.

    This ensures compatibility across different execution environments.

    Returns:
    --------
    Path
        A `Path` object representing the directory path.

    Source:
    -------
    https://www.youtube.com/@CodingIsFun
    https://www.youtube.com/watch?v=OLrC4J2-pvk&t=15s
    '''
    dir = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
    return dir


if __name__ == "__main__":
    print(_dir())  # Run this when executing hw2_util.py directly

# `Envars`
----

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path


def _envars(*args: list[str]) -> list[str] | str:
    """
    Loads environment variables from a `.env` file and retrieves specified values.

    This function attempts to load environment variables from a `.env` file 
    located in the same directory as the script. It then returns the requested 
    environment variables as either a list or a single string.

    Parameters:
    -----------
    *args : list of str
        One or more environment variable names to retrieve.

    Returns:
    --------
    list[str] | str
        - If multiple environment variable names are provided, returns a list of values.
        - If a single environment variable name is provided, returns a string.
        - Returns `None` for any variable that is not found.

    Source:
    -------
    https://www.youtube.com/@CodingIsFun
    https://www.youtube.com/watch?v=OLrC4J2-pvk&t=15s
    """
    
    try:
        # elegant pathing to current workspace
        env = _dir() / ".env"
        load_dotenv(env) # "cache" .env file
    except Exception as e:
        print(f'Error opening .env file: {str(e)}')

    # returns the requested environment variables as a list or a string
    return [os.getenv(arg) for arg in args] if len(args) > 1 else os.getenv(args[0])

# `Summation`
-----

In [ ]:
from sympy import symbols, summation

# Define the variable and expression for the summation
k, n = symbols('k n')
expression = 4*k + 1

# Calculate the summation
summation_expr = summation(expression, (k, 0, n))
# print(summation_expr.simplify())

# `Softmax`
----

In [ ]:
import math

# Given output vector
outputs = [1.2, -3.1, 4.7, 6.3, -5.2]

# Step 1: Find the maximum value
max_val = max(outputs)
print(max_val)

# Step 2: Compute the exponentials after shifting by max_val to avoid overflow
exp_values = [math.exp(x - max_val) for x in outputs]

# Step 3: Compute the sum of exponentials
sum_exp = sum(exp_values)

# Step 4: Compute softmax by dividing each exponential by the sum
softmax = [val / sum_exp for val in exp_values]

# Print the results with 3-decimal formatting
for val in softmax:
    print(f"{val:.6f}")

# `Recur Generator`
----

In [ ]:
from typing import Generator

def _train(h: list[str]) -> Generator[str, None, None]:
    '''
    Generates backoff n-grams for a given sequence.
    
    Parameters:
    -----------
    h : list of str
        A sequence of words representing an (n-1)-gram history.

    Yields:
    -------
    str
        A space-separated string representing an n-gram, progressively backed off.
        The final yield is an empty string (''), indicating no context.
    
    Example:
    --------
    >>> list(_train(["the", "quick", "brown"]))
    ['the quick brown', 'the quick', 'the', '']
    '''

    if not h: 
        yield ''
        return
    yield ' '.join(h)
    yield from _train(h[:-1])

# `Sort`
----

## `Quicksort`

In [ ]:
from typing import TypeVar
from collections.abc import Sequence

T = TypeVar("T", int, float, str)  # Supports int, float, and str

def quick_sort(a: Sequence[T]) -> list[T]:
    """
    Sorts a sequence (list or tuple) of comparable elements (int, float, or str) using the QuickSort algorithm.

    This function recursively partitions the input sequence into three parts:
    - `left`: Elements smaller than the pivot.
    - `middle`: Elements equal to the pivot.
    - `right`: Elements greater than the pivot.
    It then recursively sorts the left and right partitions before returning
    the concatenated sorted list.

    Parameters:
    -----------
    a : Sequence[T]
        A list or tuple of elements to be sorted. Elements must be of a comparable type
        (int, float, or str).

    Returns:
    --------
    list[T]
        A new sorted list containing the same elements as `a`.

    Example:
    --------
    >>> quick_sort([3.5, 1.2, 4.8, 1.1])
    [1.1, 1.2, 3.5, 4.8]

    >>> quick_sort(("pear", "apple", "orange"))
    ['apple', 'orange', 'pear']

    >>> quick_sort((5, 3, 9, 1, 7))
    [1, 3, 5, 7, 9]
    """

    if len(a) <= 1:
        return list(a)  # Convert to list to maintain return type consistency

    pivot = a[len(a) // 2]  # Select the pivot (middle element)

    left = [x for x in a if x < pivot]      # Elements smaller than pivot
    middle = [x for x in a if x == pivot]   # Elements equal to pivot
    right = [x for x in a if x > pivot]     # Elements greater than pivot

    return quick_sort(left) + middle + quick_sort(right)  # Recursively sort and combine



## `Mergesort`

In [ ]:
from typing import TypeVar
from collections.abc import Sequence

T = TypeVar("T", int, float, str)  # Supports sorting int, float, and str

def merge_sort(a: Sequence[T]) -> list[T]:
    """
    Sorts a sequence (list or tuple) of comparable elements (int, float, or str) using the Merge Sort algorithm.

    Merge Sort is a divide-and-conquer algorithm that:
    1. Recursively divides the input into two halves.
    2. Sorts each half separately.
    3. Merges the sorted halves into a single sorted sequence.

    Parameters:
    -----------
    a : Sequence[T]
        A list or tuple of elements to be sorted. Elements must be of a comparable type
        (int, float, or str).

    Returns:
    --------
    list[T]
        A new sorted list containing the same elements as `a`.

    Example:
    --------
    >>> merge_sort([3.5, 1.2, 4.8, 1.1])
    [1.1, 1.2, 3.5, 4.8]

    >>> merge_sort(("pear", "apple", "orange"))
    ['apple', 'orange', 'pear']

    >>> merge_sort((5, 3, 9, 1, 7))
    [1, 3, 5, 7, 9]

    Complexity:
    -----------
    - Average case: O(n log n)
    - Worst case: O(n log n) (stable, consistent sorting)
    """

    if len(a) <= 1:
        return list(a)  # Convert to list to maintain return type consistency

    mid = len(a) // 2  # Finding the midpoint

    # Recursively sort left and right halves
    left = merge_sort(a[:mid])
    right = merge_sort(a[mid:])

    return _merge(left, right)


def _merge(left: list[T], right: list[T]) -> list[T]:
    """Helper function to merge two sorted lists into a single sorted list."""
    sorted_list = []
    i = j = 0

    # Merge elements from left and right lists
    while i < len(left) and j < len(right):
        if left[i] < right[j]:
            sorted_list.append(left[i])
            i += 1
        else:
            sorted_list.append(right[j])
            j += 1

    # Append any remaining elements
    sorted_list.extend(left[i:])
    sorted_list.extend(right[j:])

    return sorted_list


# `Search`
----

In [ ]:
from typing import TypeVar
from collections.abc import Sequence

T = TypeVar("T", int, float, str)  # Supports sorting int and float (no str since ordering isn't meaningful)

def _partition(a: list[T], low: int, high: int) -> int:
    """
    Partitions the array around a pivot element for QuickSelect.

    This function places the pivot in its correct position, with smaller 
    elements to the left and larger elements to the right.

    Parameters:
    -----------
    a : list[T]
        The list to be partitioned.
    low : int
        The starting index of the partition range.
    high : int
        The ending index of the partition range (pivot element).

    Returns:
    --------
    int
        The index of the pivot element after partitioning.
    """
    pivot = a[high]  # Choose last element as pivot
    i = low - 1  

    for j in range(low, high):
        if a[j] < pivot:
            i += 1
            a[i], a[j] = a[j], a[i]  # Swap elements to maintain order

    a[i + 1], a[high] = a[high], a[i + 1]  # Place pivot at its correct position
    return i + 1

def quickselect(a: Sequence[T], k: int, low: int = None, high: int = None) -> T:
    """
    Finds the k-th smallest element in an unsorted list or tuple using the QuickSelect algorithm.

    QuickSelect is a variation of QuickSort that efficiently finds the k-th smallest 
    element without fully sorting the list.

    Parameters:
    -----------
    a : Sequence[T]
        A list or tuple of comparable elements (int, float).
    k : int
        The index (0-based) of the k-th smallest element.
    low : int, optional
        The lower bound for partitioning (defaults to 0).
    high : int, optional
        The upper bound for partitioning (defaults to len(a) - 1).

    Returns:
    --------
    T
        The k-th smallest element in the sequence.

    Example:
    --------
    >>> quickselect([3, 6, 8, 1, 9, 2, 5, 4, 7], 3)
    4  # 4th smallest element (0-based index)

    >>> quickselect((10.5, 2.3, 8.8, 4.7), 2)
    8.8

    Complexity:
    -----------
    - Best/Average case: O(n)
    - Worst case: O(n²) (occurs when always picking bad pivots)
    """

    # Convert tuple to list (QuickSelect modifies in place)
    a_list = list(a) if isinstance(a, tuple) else a  

    if low is None:
        low = 0

    if high is None:
        high = len(a_list) - 1

    while low <= high:
        pivot_index = _partition(a_list, low, high)

        if pivot_index == k:
            return a_list[k]  # Found k-th smallest element
        elif pivot_index < k:
            low = pivot_index + 1  # Search right half
        else:
            high = pivot_index - 1  # Search left half

    raise ValueError("k is out of bounds of the input list.")



if __name__ == "__main__":
    # Example usage
    arr = [3, 6, 8, 1, 9, 2, 5, 4, 7]
    k = 3  # Find the 3rd smallest element
    print(f"The {k}th smallest element is:", quickselect(arr, k - 1))  # Adjust for 0-based index

    # Testing with tuple
    tup = (10.5, 2.3, 8.8, 4.7)
    print(f"The 2nd smallest element in the tuple is:", quickselect(tup, 2))

    # Testing with strings
    words = ["pear", "apple", "orange", "banana", "grape"]
    print(f"The 3rd lexicographically smallest word is:", quickselect(words, 2))  # Should return 'orange'


# `Maxima Set`
----

In [ ]:
from collections.abc import Sequence

def maxima_set(coordinates: Sequence[tuple[int, int]]) -> list[tuple[int, int]]:
    """
    Computes the maxima set of a given set of 2D points.

    The maxima set consists of points that are not dominated by any other point.
    A point (x1, y1) is dominated by (x2, y2) if:
    - x1 <= x2 and y1 <= y2

    The function sorts the input lexicographically (by x, then by y), recursively partitions 
    it, and eliminates dominated points.

    Parameters:
    -----------
    coordinates : Sequence[tuple[int, int]]
        A sequence (list or tuple) of 2D points (x, y), where x and y are integers.

    Returns:
    --------
    list[tuple[int, int]]
        The maxima set, a list of non-dominated points sorted in lexicographical order.

    Example:
    --------
    >>> maxima_set([(8, 4), (7, 6), (9, 1), (4, 5), (6, 8), (5, 3), (3, 7)])
    [(6, 8), (7, 6), (8, 4), (9, 1)]

    Complexity:
    -----------
    - Best case: O(n log n) (due to sorting)
    - Worst case: O(n log n) (divide & conquer recursion)
    """ 

    if not all(isinstance(coord, tuple) and len(coord) == 2 for coord in coordinates):
        raise TypeError("All elements in 'coordinates' must be tuples of length 2 (x, y).")

    if len(coordinates) <= 1:
        return list(coordinates)  # Convert to list for consistency

    sorted_coords = sorted(coordinates, key = lambda point: (point[0], point[1]))

    mid = len(sorted_coords) // 2

    L = sorted_coords[:mid]  # Left half
    R = sorted_coords[mid:]  # Right half

    M1 = maxima_set(L)
    M2 = maxima_set(R)

    q = min(M2, key = lambda point: (point[0], point[1]))

    M1 = [r for r in M1 if r[0] > q[0] or r[1] > q[1]]

    return M1 + M2


if __name__ == "__main__":
    test_cases = [
        [(8, 4), (7, 6), (9, 1), (4, 5), (6, 8), (5, 3), (3, 7)],
        [(1, 1), (2, 2), (3, 3), (4, 4)],
        [(5, 5), (2, 8), (9, 3), (7, 7), (6, 9)],
        [(10, 10), (1, 1), (5, 6), (8, 2)],
        [(5, 5), [2, 8], (9, 3), (7, 7), (6, 9)],  # Invalid test case (list instead of tuple)
    ]

    for i, coords in enumerate(test_cases, 1):
        try:
            maxima = maxima_set(coords)
            print(f"Test {i} : {maxima}")
        except TypeError as e:
            print(f"Test {i} Failed: {e}")